# Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#%cd /content/gdrive/'My Drive'/'My DeepLearning'/
#!unzip 'Cat and Dog Dataset.zip'

# Importing Libraries

In [2]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications


# Importing Dataset and Splitting the Dataset to Traning Set and Validation


In [3]:
path = "//content//gdrive//My Drive//My DeepLearning//"
import os
os.path.isdir(path)   

True

In [4]:
path

'//content//gdrive//My Drive//My DeepLearning//'

In [5]:
# image dimensions
img_width, img_height = 150, 150
# top_model_weights_path = path + 'Transfer Learning Models//bottleneck_fc_model.h5'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

In [60]:
import os

#base_dir = path + "Cat and Dog Dataset"
base_dir = os.path.join(path,"Cat and Dog Dataset")

#model_dir = path + "TransferLearningModels"
model_dir = os.path.join(path,"Transfer Learning Models")

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

test=os.path.join(base_dir,'test')

In [52]:
model_dir

'//content//gdrive//My Drive//My DeepLearning//Transfer Learning Models'

In [ ]:
base_dir

'//content//gdrive//My Drive//My DeepLearning//Cat and Dog Dataset'

In [48]:
train_cat_fnames = os.listdir(train_cats_dir)
train_cat_fnames.sort()
print(train_cat_fnames[:20])

train_dog_fnames = os.listdir(train_dogs_dir)
train_dog_fnames.sort()
print(train_dog_fnames[:10])

['cat.0.jpg', 'cat.1.jpg', 'cat.10.jpg', 'cat.100.jpg', 'cat.101.jpg', 'cat.102.jpg', 'cat.103.jpg', 'cat.104.jpg', 'cat.105.jpg', 'cat.106.jpg', 'cat.107.jpg', 'cat.108.jpg', 'cat.109.jpg', 'cat.11.jpg', 'cat.110.jpg', 'cat.111.jpg', 'cat.112.jpg', 'cat.113.jpg', 'cat.114.jpg', 'cat.115.jpg']
['dog.0.jpg', 'dog.1.jpg', 'dog.10.jpg', 'dog.100.jpg', 'dog.101.jpg', 'dog.102.jpg', 'dog.103.jpg', 'dog.104.jpg', 'dog.105.jpg', 'dog.106.jpg']


In [78]:
print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))

total training cat images: 1000
total training dog images: 1000
total validation cat images: 500
total validation dog images: 500


In [84]:
training_data_dir = train_dir
validation_data_dir = validation_dir
test_data_dir = test

# Model

In [9]:
# build the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet')

#weights: lay trong so trong imagenet 
#include_top=False : chi trích Feature Map, FC tự train

# Extracting features

In [90]:
datagen = ImageDataGenerator(rescale=1. / 255)
generator = datagen.flow_from_directory(
     training_data_dir,
     target_size=(img_width, img_height),
     batch_size=batch_size,
     class_mode=None,
     shuffle=False)
bottleneck_features_train = model.predict_generator(generator, nb_train_samples // batch_size)

np.save(open(model_dir + '/bottleneck_features_train.npy','wb'),bottleneck_features_train)


Found 2000 images belonging to 2 classes.


InvalidArgumentError: ignored

In [ ]:
generator = datagen.flow_from_directory(
     validation_data_dir,
     target_size=(img_width, img_height),
     batch_size=batch_size,
     class_mode=None,
     shuffle=False)
bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples // batch_size)
np.save(open(model_dir + '/bottleneck_features_validation_A.npy', 'wb'),
         bottleneck_features_validation

# Fully Connected


In [ ]:
train_data = np.load(open(os.path.join(model_dir, 'bottleneck_features_train.npy'), 'rb'))
train_labels = np.array(
    [0] * 1000 + [1] * 1000)

validation_data = np.load(open(model_dir + '//bottleneck_features_validation.npy', 'rb'))
validation_labels = np.array(
    [0] * 400 + [1] * 400)

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 2,097,665
Trainable params: 2,097,665
Non-trainable params: 0
_________________________________________________________________


# Save Model

In [12]:
model.save_weights(model_dir + "//model_Cat and Dog Classifier VGG16.h5")

# Testing Model

In [13]:
MODEL_FILE = model_dir + "//model_Cat and Dog Classifier VGG16.h5"
# Load Model
model.load_weights(MODEL_FILE)

In [ ]:
test_data_generator = ImageDataGenerator(rescale=1./255)
test_generator = test_data_generator.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=1,
    class_mode="binary",
    shuffle=False)

In [ ]:
test

In [72]:
# build the VGG16 network
Pre_model = applications.VGG16(include_top=False, weights='imagenet')

In [87]:
batch_size = 50
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

bottleneck_features = Pre_model.predict_generator(test_generator, batch_size)
probabilities = model.predict(bottleneck_features,batch_size)

In [ ]:
for index, probability in enumerate(probabilities):
    image_path = test_data_dir+ "/" +test_generator.filenames[index]
    img = mpimg.imread(image_path)
    plt.imshow(img)
    if probability > 0.5:
        plt.title("%.2f" % (probability[0]*100) + "% dog")
    else:
        plt.title("%.2f" % ((1-probability[0])*100) + "% cat")
    plt.show()